<h1> Eploring the Zillow prize dataset </h1>

In this notebook we are going to explore the provided datasets in order to better understand the problem at hand. At the same time we will try to arrive at preliminary conclusions on the distribution of our data in order to derive useful insights for the modelling phase.

Lets start by loading the training dataset for inspection:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore') # Bad practice my ass

train_features = pd.read_csv('data/train_features.csv')

<h3> Renaming the columns </h3>

It is obvious that these column names are hard to interpret, we will therefore use a custom mapping. Since
this falls more into the data layer than the analysis we will read the mapping from a file. The expected format
is a key value pair per line split by the "=" symbol. 

For example: 

`newColumnName = oldColumnName`

In [ ]:
col_mapping = pd.read_csv('data/feature_names', sep ="=", header=None).applymap(str.strip)
mapping_dict = dict(zip(col_mapping[1], col_mapping[0]))
train_features.rename(columns = mapping_dict, inplace=True)

(num_records, num_features) = train_features.shape
print('There are {0} properties recorded and {1} features in total'.format(*train_features.shape))

<h3> Handling sparse columns </h3>

There are quite a lot of features, however some of them have a lot of missing values. Lets try to quantify that statement

In [ ]:
nan_count = train_features.isnull().mean()

threshold = 0.99
print('Some features with more than {0}% NaN values: \n'.format(threshold * 100))
print(nan_count[nan_count > threshold].sort_values(ascending=False))

plot = sns.distplot(nan_count * 100, hist=False, rug=True)
plot.set(xlim=(0, 100), yticks=[], ylabel='Features', xlabel='NaN %')
sns.plt.title('NaN Percentage', weight='bold')
plt.show()

<h4> More than 1/3 of our features are sparse! </h4>

Several of these sparse features could be important such as the tax delinquency. In others however a missing value can be easily imputed. For example missing values for the pool type and area probably mean that the property does not have a pool installed. Moreover the histogram seems inverserly Guassian, this means that features are either mainly blank (right side of the curve) or almost always filled (left side).

<i>These missing values may or may not pose a significant problem at the modelling phase</i>

<h3> Checking the target variable </h3>

Lets now take a look at our target variable: <b> The log Error </b>

In [ ]:
labels = pd.read_csv('data/train_label.csv')

# Rename is needed to facilitate a left join
labels.rename(columns={'parcelid': 'ID'}, inplace=True)

# What part of properties have an associated label?
print('Only ' + '{0:.2f}'.format(len(labels) * 100 / len(train_features)) + '% properties are labeled')

<h3> A lot of missing labels </h3>

Well it seems that most of the properties found in our feature dataset do not have a label associated with them.
This could probably come from the fact that a prediction error can only be computed when the real selling price is recorded, that is when the property is actually sold. Of course not every property was sold within the time limits of the data collection. This will significantly reduce our training set since we will only keep records where the label is known for the modelling phase.

In [ ]:
# Merge the 2 datasets, only keeping properties for which the target variable is known
merged = train_features.merge(labels, on='ID', how='inner')

<h3> Univariate analysis </h3>

Lets start by identifying features that are correlated to the target variable. Note that this only shows 
linear relations between each feature and the target variable (and not combinations of features).

In [ ]:
def cross_corr(feature):
    target = merged['logerror']
    if feature.isnull().mean() > 0.2:
        # Too many NaN, imputing does not make sense
        return None
    try:
        imputed_feature = feature.fillna(feature.mean())
        return np.corrcoef(imputed_feature, target)[0, 1]
    except TypeError:
        # Cannot correlate categorical features
        return None
    
# Lets find the cross correlation of every numerical feature to the target variable
coeffs = merged.drop(['ID', 'logerror'], axis=1).apply(cross_corr)
print('These features show the highest correlation (in absolute value) to the target variable: \n')
print(coeffs.dropna().abs().sort_values(ascending=False).head(5))

# Lets plot for easier inspection
univariate = pd.DataFrame({'features': coeffs.index, 'correlation coef.': coeffs.values})
plot = sns.barplot('features', 'correlation coef.', data=univariate)
plot.set(ylim=(coeffs.min(), coeffs.max()), ylabel='pearson coeff.', title='Univariate Correlation')
plot.set_xticklabels(coeffs.index, rotation=90)#if this is confusing comment it out
sns.plt.show()

<h3> Univariate analysis not too promising </h3>

Observations:
- Only a subset of our features have a meaningful correlation to the target value (for example categorical one's or numerical with many missing values do not).

- Most features are positively correlated to he target (higher values lead to higher logerror)

- All in all correlations are very weak (< 0.1). This implies that simple linear models will probably fail to capture the actual relationship of the input to the logerror.


<h3> Dimensionality Reduction </h3>

Another observation is that we have a rather large number of available features. We would expect some of them to
be either correlated to others, or even reduntant alltogether. It might make sense to reduce the dimensions using PCA, and check what percentage of our features are really essential.

**Since PCA only handles numerical data we will focus on these only.**

In [ ]:
from sklearn.decomposition import PCA

# PCA is unsupervised so we should ignore the target variable. It also works with numerical features only
def fits_PCA(col, df=merged):
    def is_dense(threshold=0.99):
        return df[col].isnull().mean() < threshold
    def is_numerical():
        return df[col].dtype in ['float64', 'int64']
    def is_label():
        return col in ['logerror', 'ID']
    def is_constant():
        return df[col].max() == df[col].min()

    return is_dense() and is_numerical() and not is_label() and not is_constant()

def normalize(df):
    return (df - df.mean()) / (df.max() - df.min())

# Pick fitting columns for running PCA 
PCA_cols = [col for col in merged.columns if fits_PCA(col)]

# Subset fitting columns, fill missing values and normalize
pca_df = normalize(merged[PCA_cols].fillna(merged[PCA_cols].median()))

pca = PCA()
pca.fit(pca_df)
plot = sns.tsplot(np.cumsum(pca.explained_variance_ratio_))
plot.set(ylabel='% variance explained', xlabel='number of features', title='PCA')
plt.show()

<h4> We can safely reduce our features by a factor of 2! </h4>

It is obvious from the above figure that we can reach almost 99% of the total variability using only 15 out of 38 numerical features.
We can get even greater reduction if we are willing to miss on some information, perhaps using 10 features and retaining more than 95% of the total variance.

<h4> Store necessary data for later analysis </h4>
This file so far is the first exprolation of the data. We need to store any usefull data for later analysis. Such as the final merged .csv file and the coefficients.

In [ ]:
merged.to_csv('data/merged.csv', index=False)
coeffs.to_csv('data/coeffs.csv')

# Let's identify categorical variables

In [ ]:
categorical_var_ID=['HeatingOrSystemTypeID','PropertyLandUseTypeID','StoryTypeID','AirConditioningTypeID','ArchitecturalStyleTypeID','TypeConstructionTypeID','BuildingClassTypeID']
categorical_var_Location=['regionidcounty','regionidcity','regionidneighborhood']